# VacationPy
----
### Observations 
After gathering and analyzing api data on weather conditions: temperture, humidity, cloud cover, wind speed, and geolocation (latitudinal distance from the equator) for 565 cities in both the northern (384) and southern (181) hemisphere for the month of Febuary 2022, there seems to be a strongly negative correlation between temperature (in degrees Fahrenheit) and geolocation from the equator for cities in the northern hemisphere and a slightly positive correlation for the same metrics for cities located in the southern hemisphere. The farther away a city is located from the equator (0 degrees latitude) the colder the climate, mainly for the ranges beyond the range of -20 to 20 (degrees latitude). 

There seems to be little to no statistically relevent correlation between humidity, cloudiness, and wind speed with geolocation in respects to the equator as the data points are spreaded throughout and does not form a logical pattern that when not skewed by server outliers. Wind speed between over 30 mph can be found for a specific cities in the northern hemisphere located between 60 and 70 (degrees latitude) such as Sørland (67.6670, 32.26). There are also outliers like Isangel (40.18, -19.5500) and Vao (-22.6667, 30.29) for the southern hemisphere with also with over 30 mph wind speeds.   

There is no observable correlation for cloudiness (cloud cover) and geolocation for all cities (both north and south of the equator). The data points appear to be randomly scattered all through out the linear regression plot with no obvious association that can be analyzed.  




In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import csv from WeatherPy 
weather_csv = "../WeatherPy/output_data/cities.csv"

weather_df = pd.read_csv(weather_csv)

weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Airai,-8.9266,125.4092,67.93,80,99,0.98,TL,1644451500
1,1,Viedma,-40.8135,-62.9967,82.04,37,94,6.08,AR,1644451501
2,2,Port Alfred,-33.5906,26.8910,65.64,85,5,9.98,ZA,1644451367
3,3,Cayenne,4.9333,-52.3333,76.08,91,75,1.01,GF,1644451275
4,4,Opuwo,-18.0607,13.8400,66.49,87,100,5.93,NaN,1644451501
...,...,...,...,...,...,...,...,...,...,...
567,567,Qaqortoq,60.7167,-46.0333,16.30,71,95,9.95,GL,1644451639
568,568,Hastings,50.8552,0.5729,49.37,85,95,9.33,GB,1644451639
569,569,Plyussa,58.4308,29.3654,34.14,97,100,11.88,RU,1644451639
570,570,Quemado de Güines,22.7853,-80.2536,71.82,89,81,10.45,CU,1644451639


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Config gmaps with api key.
gmaps.configure(api_key=g_key)

# Drop NA
weather_df = weather_df.dropna()

# Convert to Float
humidity = weather_df["Humidity"].astype(float)

# Storage 
location = weather_df[["Lat", "Lng"]].astype(float)

# Humidity Heatmap Layer
figure = gmaps.figure()

heat = gmaps.heatmap_layer(location, weights = humidity, dissipating = False, max_intensity = max(weather_df["Humidity"]), point_radius = 2)

figure.add_layer(heat)

figure

Figure(layout=FigureLayout(height='420px'))

In [4]:
# New DataFrame
ideal_weather_df = weather_df

"""
Conditionals: 

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
"""
# A max temperature lower than 80 degrees but higher than 70.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop NA
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,Coihueco,-36.6167,-71.8333,73.90,24,0,5.01,CL,1644451522
141,141,Inhambane,-23.8650,35.3833,75.20,82,0,4.65,MZ,1644451532
283,283,Cerro Largo,-28.1486,-54.7381,72.00,36,0,4.32,BR,1644451566
347,347,Laguna,38.4210,-121.4238,78.66,42,0,4.61,US,1644451582
421,421,De Aar,-30.6497,24.0123,72.57,22,0,7.29,ZA,1644451601
430,430,Geraldton,-28.7667,114.6000,70.21,64,0,9.22,AU,1644451603
494,494,Salinas,36.6777,-121.6555,76.37,38,0,5.75,US,1644451619
554,554,Tomatlán,19.9333,-105.2500,79.70,36,0,9.86,MX,1644451635


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_map_df = ideal_weather_df

# params, iteration, etc.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# iteration 
for index, row in hotel_map_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Result for the Index {index}: {row['City']}.")
    
    # response
    response = requests.get(url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f"Nearest hotel is {results[0]['name']}.")
        hotel_map_df.loc[index, "Name of Hotel"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Skip Over Missing Info...")
        
    # print delimiter 
    print("---------------------------------")
# 10 results 

Result for the Index 92: Coihueco.
Nearest hotel is Frutillares Centro Coihueco.
---------------------------------
Result for the Index 141: Inhambane.
Nearest hotel is Oceano.
---------------------------------
Result for the Index 283: Cerro Largo.
Nearest hotel is Hotel Missões.
---------------------------------
Result for the Index 347: Laguna.
Nearest hotel is Holiday Inn Express & Suites Elk Grove Central - Hwy 99, an IHG Hotel.
---------------------------------
Result for the Index 421: De Aar.
Nearest hotel is Nyibiba Guesthouse.
---------------------------------
Result for the Index 430: Geraldton.
Nearest hotel is Broadwater Mariner Resort.
---------------------------------
Result for the Index 494: Salinas.
Nearest hotel is Laurel Inn & Conference Center.
---------------------------------
Result for the Index 554: Tomatlán.
Nearest hotel is El Salvial.
---------------------------------


In [6]:
hotel_map_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Name of Hotel
92,92,Coihueco,-36.6167,-71.8333,73.90,24,0,5.01,CL,1644451522,Frutillares Centro Coihueco
141,141,Inhambane,-23.8650,35.3833,75.20,82,0,4.65,MZ,1644451532,Oceano
283,283,Cerro Largo,-28.1486,-54.7381,72.00,36,0,4.32,BR,1644451566,Hotel Missões
347,347,Laguna,38.4210,-121.4238,78.66,42,0,4.61,US,1644451582,Holiday Inn Express & Suites Elk Grove Central...
421,421,De Aar,-30.6497,24.0123,72.57,22,0,7.29,ZA,1644451601,Nyibiba Guesthouse
430,430,Geraldton,-28.7667,114.6000,70.21,64,0,9.22,AU,1644451603,Broadwater Mariner Resort
494,494,Salinas,36.6777,-121.6555,76.37,38,0,5.75,US,1644451619,Laurel Inn & Conference Center
554,554,Tomatlán,19.9333,-105.2500,79.70,36,0,9.86,MX,1644451635,El Salvial


In [7]:
# Add marker layer ontop of heat map
locations = hotel_map_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name of Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_map_df.iterrows()]
locations = hotel_map_df[["Lat", "Lng"]]

# Symbol Layer
hotel_map_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_info)

# Additional Layers
figure.add_layer(markers)
figure.add_layer(hotel_map_layer)

# Display figure

figure

Figure(layout=FigureLayout(height='420px'))